In [ ]:
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.models import VGG16_Weights
import zipfile

In [ ]:
pip install torchinfo

In [ ]:
##########################
### SETTINGS
##########################

RANDOM_SEED = 123
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
with zipfile.ZipFile("archive (8).zip", "r") as zip_ref:
    zip_ref.extractall("Satellite")

In [ ]:
data_directory = pathlib.Path('/content/Satellite')
#reverse folder order
class_names = [item.name for item in data_directory.glob('*')][:2][::-1]
print(class_names)

In [ ]:
data_dir = '/content/Satellite/data'

cloudy_img = '/content/Satellite/data/cloudy'
desert_img = '/content/Satellite/data/desert'
green_area_img = '/content/Satellite/data/green_area'
water_img = '/content/Satellite/data/water'

print(f'Number of cloudy images:      {len(os.listdir(cloudy_img))}')
print(f'Number of desert images:      {len(os.listdir(desert_img))}')
print(f'Number of green_area images:  {len(os.listdir(green_area_img))}')
print(f'Number of water images:       {len(os.listdir(water_img))}')

# **Data Augmentation**

In [ ]:
import torchvision.transforms as transforms

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
data = ImageFolder(data_dir, transform=None)

In [ ]:
from torch.utils.data import random_split

train_len = int(0.80 * len(data))
val_len   = int(0.10 * len(data))
test_len  = len(data) - train_len - val_len
print(train_len,val_len,test_len)

generator = torch.Generator().manual_seed(RANDOM_SEED)
train_data, val_data, test_data = random_split(data, [train_len, val_len, test_len], generator=generator)

In [ ]:
from torch.utils.data import ConcatDataset, Subset
import copy

augmentation_factor = 4

augmented_images = []

for i in range(augmentation_factor):
    train_subset = Subset(data, train_data.indices)
    train_subset.dataset = copy.deepcopy(train_subset.dataset)
    train_subset.dataset.transform = train_transforms

    augmented_images.append(train_subset)

augmented_train_data = ConcatDataset(augmented_images)
val_data.dataset.transform = val_test_transforms
test_data.dataset.transform = val_test_transforms

In [ ]:
batch_size = 32
train_loader = DataLoader(augmented_train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=batch_size, shuffle=False)
val_loader   = DataLoader(val_data,  batch_size=batch_size, shuffle=False)

In [ ]:
print(data.class_to_idx)
num_classes = len(data.class_to_idx)
print(num_classes)

In [ ]:
print(f"Total images in dataset: {len(data)}")
print(f"Train split size: {len(train_data)}")
print(f"Validation split size: {len(val_data)}")
print(f"Test split size: {len(test_data)}")
print(f"Augmented train size: {len(augmented_train_data)}")

In [ ]:
from collections import Counter


val_labels = [val_data[i][1] for i in range(len(val_data))]
test_labels = [test_data[i][1] for i in range(len(test_data))]

val_class_counts = Counter(val_labels)
test_class_counts = Counter(test_labels)

print("Validation class distribution:", val_class_counts)
print("Test class distribution:", test_class_counts)

In [ ]:
aug_labels = [augmented_train_data[i][1] for i in range(len(augmented_train_data))]
aug_class_counts = Counter(aug_labels)

print("Augmented Train class distribution:", aug_class_counts)


# **VGG 16**

In [ ]:
from torchvision import models
from torchvision.models import VGG16_Weights
from torchsummary import summary
import torch

model = models.vgg16(weights=VGG16_Weights.DEFAULT)
summary(model, (3, 224, 224))


## Tuned 1

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import time


model = models.vgg16(weights=VGG16_Weights.DEFAULT)

model.classifier[2] = nn.Dropout(p=0.4)
model.classifier[5] = nn.Dropout(p=0.4)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Use GPU
model = model.to(device)


#loss function and optimization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

train_accuracies = []
val_accuracies = []



start_time = time.time()

num_epochs = 10
for epoch in range(num_epochs):


  #Training
  model.train()
  train_loss = 0.0
  train_preds = [] #preds
  actual_labels = [] #actual label


  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    train_preds.extend(outputs.argmax(1).cpu().numpy())
    actual_labels.extend(labels.cpu().numpy())

  train_acc = accuracy_score(actual_labels, train_preds)
  avg_train_loss = train_loss / len(train_loader)

  #Validation
  model.eval()
  val_loss = 0.0
  val_preds = []
  val_labels = []

  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, labels)

      val_loss += loss.item()
      val_preds.extend(outputs.argmax(1).cpu().numpy())
      val_labels.extend(labels.cpu().numpy())

  val_acc = accuracy_score(val_labels, val_preds)
  avg_val_loss = val_loss / len(val_loader)

  train_accuracies.append(train_acc)
  val_accuracies.append(val_acc)


  print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")



end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

train_acc_vgg = np.array(train_accuracies)
val_acc_vgg = np.array(val_accuracies)

print("\nFinal Results:")
print(f"Average Training Accuracy over: {train_acc_vgg.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_vgg.mean():.4f}")



# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')





plt.figure(figsize=(10, 6))
plt.plot(train_acc_vgg, label='Training Accuracy', marker='o')
plt.plot(val_acc_vgg, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



## Best Model

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import time


model = models.vgg16(weights=VGG16_Weights.DEFAULT)

model.classifier[2] = nn.Dropout(p=0.5)
model.classifier[5] = nn.Dropout(p=0.5)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

model = model.to(device)


#loss function and optimization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)

train_accuracies = []
val_accuracies = []


start_time = time.time()

num_epochs = 10
for epoch in range(num_epochs):

  #Training
  model.train()
  train_loss = 0.0
  train_preds = [] #preds
  actual_labels = [] #actual label


  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    train_preds.extend(outputs.argmax(1).cpu().numpy())
    actual_labels.extend(labels.cpu().numpy())

  train_acc = accuracy_score(actual_labels, train_preds)
  avg_train_loss = train_loss / len(train_loader)

  #Validation
  model.eval()
  val_loss = 0.0
  val_preds = []
  val_labels = []

  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, labels)

      val_loss += loss.item()
      val_preds.extend(outputs.argmax(1).cpu().numpy())
      val_labels.extend(labels.cpu().numpy())

  val_acc = accuracy_score(val_labels, val_preds)
  avg_val_loss = val_loss / len(val_loader)

  train_accuracies.append(train_acc)
  val_accuracies.append(val_acc)


  print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")





end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

train_acc_vgg = np.array(train_accuracies)
val_acc_vgg = np.array(val_accuracies)

print("\nFinal Results:")
print(f"Average Training Accuracy over: {train_acc_vgg.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_vgg.mean():.4f}")



# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_acc_vgg, label='Training Accuracy', marker='o')
plt.plot(val_acc_vgg, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
torch.save(model.state_dict(), 'best_vgg16_model_3.pth')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=test_loader.dataset.dataset.classes
)
disp.plot(xticks_rotation=45)
plt.title("Confusion Matrix - Test Set")
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import random

fractions = [0.2, 0.4, 0.6, 0.8, 1.0]
num_epochs = 5
batch_size = 32
results = []

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Full training set
train_indices = list(range(len(train_data)))
random.shuffle(train_indices)

for frac in fractions:
    print(f"\nTraining on {int(frac * 100)}% of the dataset...")

    split_len = int(frac * len(train_indices))
    subset_indices = train_indices[:split_len]
    train_subset = Subset(train_data, subset_indices)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

    # Model
    model = models.vgg16(weights=VGG16_Weights.DEFAULT)
    model.classifier[2] = nn.Dropout(0.5)
    model.classifier[5] = nn.Dropout(0.5)
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)

    # Train
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    #testing
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    results.append(acc)
    print(f"Test Accuracy with {int(frac*100)}% data: {acc:.4f}")

plt.figure(figsize=(8,5))
plt.plot([int(f*100) for f in fractions], results, marker='o')
plt.xlabel("Training Dataset Size (%)")
plt.ylabel("Test Accuracy")
plt.title("Test Accuracy vs Dataset Size")
plt.grid(True)
plt.show()


## Tuned 3

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import time


model = models.vgg16(weights=VGG16_Weights.DEFAULT)

model.classifier[2] = nn.Dropout(p=0.6)
model.classifier[5] = nn.Dropout(p=0.6)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Use GPU
model = model.to(device)


#loss function and optimization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)

train_accuracies = []
val_accuracies = []


start_time = time.time()

num_epochs = 20
for epoch in range(num_epochs):



    #Training
  model.train()
  train_loss = 0.0
  train_preds = [] #preds
  actual_labels = [] #actual label


  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    train_preds.extend(outputs.argmax(1).cpu().numpy())
    actual_labels.extend(labels.cpu().numpy())

  train_acc = accuracy_score(actual_labels, train_preds)
  avg_train_loss = train_loss / len(train_loader)

  #Validation
  model.eval()
  val_loss = 0.0
  val_preds = []
  val_labels = []

  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, labels)

      val_loss += loss.item()
      val_preds.extend(outputs.argmax(1).cpu().numpy())
      val_labels.extend(labels.cpu().numpy())

  val_acc = accuracy_score(val_labels, val_preds)
  avg_val_loss = val_loss / len(val_loader)

  train_accuracies.append(train_acc)
  val_accuracies.append(val_acc)


  print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")



end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

train_acc_vgg = np.array(train_accuracies)
val_acc_vgg = np.array(val_accuracies)

print("\nFinal Results:")
print(f"Average Training Accuracy over: {train_acc_vgg.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_vgg.mean():.4f}")



# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_acc_vgg, label='Training Accuracy', marker='o')
plt.plot(val_acc_vgg, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# **Resnet**

In [ ]:
from torchvision.models import ResNet18_Weights

model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
summary(model, (3, 224, 224))


## Best Model

In [ ]:
from torchvision.models import ResNet18_Weights
import time

model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, num_classes)


model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)


train_accuracies = []
val_accuracies = []

start_time = time.time()

num_epochs = 10
for epoch in range(num_epochs):

    #Training
    model.train()
    train_loss = 0.0
    train_preds = [] #preds
    actual_labels = [] #actual label


    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    #Validation
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)


    print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")


end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

train_acc_resnet = np.array(train_accuracies)
val_acc_resnet = np.array(val_accuracies)


print(f"Average Training Accuracy over: {train_acc_resnet.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_resnet.mean():.4f}")


# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')

In [ ]:
torch.save(model.state_dict(), 'best_ResNet_model_1.pth')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_acc_resnet, label='Training Accuracy', marker='o')
plt.plot(val_acc_resnet, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=test_loader.dataset.dataset.classes
)
disp.plot(xticks_rotation=45)
plt.title("Confusion Matrix - Test Set")
plt.show()


In [ ]:
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import accuracy_score
from torch.utils.data import Subset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import time

# Parameters
fractions = [0.2, 0.4, 0.6, 0.8, 1.0]
num_epochs = 5
batch_size = 32
results = []

random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Shuffle training indices
train_indices = list(range(len(train_data)))
random.shuffle(train_indices)

for frac in fractions:
    print(f"\nTraining on {int(frac * 100)}% of the dataset...")

    # Prepare subset
    split_len = int(frac * len(train_indices))
    subset_indices = train_indices[:split_len]
    train_subset = Subset(train_data, subset_indices)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

    #model
    model = resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    # Training
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Testing
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    results.append(acc)
    print(f"Test Accuracy with {int(frac * 100)}% data: {acc:.4f}")


plt.figure(figsize=(8,5))
plt.plot([int(f*100) for f in fractions], results, marker='o')
plt.xlabel("Training Dataset Size (%)")
plt.ylabel("Test Accuracy")
plt.title("ResNet18 - Test Accuracy vs Dataset Size")
plt.grid(True)
plt.show()


## Tuned Model 1

In [ ]:
from torchvision.models import ResNet18_Weights
import time

model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(model.fc.in_features, num_classes)
)


model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)


train_accuracies = []
val_accuracies = []

start_time = time.time()
num_epochs = 10
for epoch in range(num_epochs):

    #Training
    model.train()
    train_loss = 0.0
    train_preds = [] #preds
    actual_labels = [] #actual label


    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    #Validation
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)


    print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

train_acc_resnet = np.array(train_accuracies)
val_acc_resnet = np.array(val_accuracies)

print(f"Average Training Accuracy over: {train_acc_resnet.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_resnet.mean():.4f}")


# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_acc_resnet, label='Training Accuracy', marker='o')
plt.plot(val_acc_resnet, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Tune model 2

In [ ]:
from torchvision.models import ResNet18_Weights

model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(model.fc.in_features, num_classes)
)


model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


train_accuracies_1 = []
val_accuracies_1 = []

num_epochs = 17
for epoch in range(num_epochs):

    #Training
    model.train()
    train_loss = 0.0
    train_preds = [] #preds
    actual_labels = [] #actual label


    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    #Validation
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies_1.append(train_acc)
    val_accuracies_1.append(val_acc)


    print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")


train_acc_resnet = np.array(train_accuracies_1)
val_acc_resnet = np.array(val_accuracies_1)

print(f"Average Training Accuracy over: {train_acc_resnet.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_resnet.mean():.4f}")


# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_acc_resnet, label='Training Accuracy', marker='o')
plt.plot(val_acc_resnet, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# **Dense Net**

In [ ]:
import torch
from torchvision import models
from torchvision.models import DenseNet121_Weights
from torchinfo import summary

model = models.densenet121(weights=DenseNet121_Weights.DEFAULT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

summary(model, input_size=(1, 3, 224, 224))


## Best Model

In [ ]:
from torchvision.models import DenseNet121_Weights
import time

model = models.densenet121(weights=DenseNet121_Weights.DEFAULT)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(model.classifier.in_features, num_classes)
)
model = model.to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.01)

# Track accuracy
train_accuracies = []
val_accuracies = []

start_time = time.time()
num_epochs = 10



for epoch in range(10):
    model.train()
    train_loss = 0.0
    train_preds = []
    actual_labels = []

    #training
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")


end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Final results
train_acc_arr = np.array(train_accuracies)
val_acc_arr = np.array(val_accuracies)

print("\nFinal Results:")
print(f"Average Training Accuracy: {train_acc_arr.mean():.4f}")
print(f"Average Validation Accuracy: {val_acc_arr.mean():.4f}")


# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=test_loader.dataset.dataset.classes
)
disp.plot(xticks_rotation=45)
plt.title("Confusion Matrix - Test Set")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_acc_arr, label='Training Accuracy', marker='o')
plt.plot(val_acc_arr, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)  # Set y-axis from 0 to 1
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
torch.save(model.state_dict(), 'best_DenseNet_model_2.pth')

In [ ]:
from torchvision.models import densenet121, DenseNet121_Weights
from sklearn.metrics import accuracy_score
from torch.utils.data import Subset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import time

# Parameters
fractions = [0.2, 0.4, 0.6, 0.8, 1.0]
num_epochs = 5
batch_size = 32
results = []

# Fix random seed
random.seed(42)
torch.manual_seed(42)

# Shuffle indices of the full train set
train_indices = list(range(len(train_data)))
random.shuffle(train_indices)

for frac in fractions:
    print(f"\nTraining on {int(frac * 100)}% of the dataset...")

    subset_len = int(frac * len(train_indices))
    subset_indices = train_indices[:subset_len]
    train_subset = Subset(train_data, subset_indices)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

    # Model
    model = densenet121(weights=DenseNet121_Weights.DEFAULT)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(model.classifier.in_features, num_classes)
    )
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    # Training
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Testing
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    results.append(acc)
    print(f"Test Accuracy with {int(frac * 100)}% data: {acc:.4f}")


plt.figure(figsize=(8, 5))
plt.plot([int(f*100) for f in fractions], results, marker='o')
plt.xlabel("Training Dataset Size (%)")
plt.ylabel("Test Accuracy")
plt.title("DenseNet121 - Test Accuracy vs Dataset Size")
plt.grid(True)
plt.tight_layout()
plt.show()


## Tune 2

In [ ]:
from torchvision.models import DenseNet121_Weights
import time

model = models.densenet121(weights=DenseNet121_Weights.DEFAULT)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(model.classifier.in_features, num_classes)
)
model = model.to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

# Track accuracy
train_accuracies = []
val_accuracies = []

start_time = time.time()
num_epochs = 10


# Training
for epoch in range(10):
    model.train()
    train_loss = 0.0
    train_preds = []
    actual_labels = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")


end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Final results
train_acc_arr = np.array(train_accuracies)
val_acc_arr = np.array(val_accuracies)

print("\nFinal Results:")
print(f"Average Training Accuracy: {train_acc_arr.mean():.4f}")
print(f"Average Validation Accuracy: {val_acc_arr.mean():.4f}")


# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_acc_arr, label='Training Accuracy', marker='o')
plt.plot(val_acc_arr, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Tuned 3

In [ ]:
model = models.densenet121(weights=DenseNet121_Weights.DEFAULT)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(model.classifier.in_features, num_classes)
)
model = model.to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay= 0.001)

# Track accuracy
train_accuracies = []
val_accuracies = []

start_time = time.time()
num_epochs = 10

# Training
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_preds = []
    actual_labels = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Final results
train_acc_arr = np.array(train_accuracies)
val_acc_arr = np.array(val_accuracies)

print("\nFinal Results:")
print(f"Average Training Accuracy: {train_acc_arr.mean():.4f}")
print(f"Average Validation Accuracy: {val_acc_arr.mean():.4f}")


# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_acc_arr, label='Training Accuracy', marker='o')
plt.plot(val_acc_arr, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# GoogLeNet

In [ ]:
from torchvision.models import GoogLeNet_Weights

model = models.googlenet(weights=GoogLeNet_Weights.DEFAULT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
summary(model, (3, 224, 224))


## Best Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import GoogLeNet_Weights
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import time

model = models.googlenet(weights=GoogLeNet_Weights.DEFAULT)
model.fc = nn.Sequential(nn.Dropout(p=0.5),nn.Linear(model.fc.in_features, num_classes))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay = 0.001)

train_accuracies = []
val_accuracies = []

start_time = time.time()
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_preds = []
    actual_labels = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

train_acc_google = np.array(train_accuracies)
val_acc_google = np.array(val_accuracies)

print(f"\nFinal Results:")
print(f"Average Training Accuracy over: {train_acc_google.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_google.mean():.4f}")

# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_accuracies, label='Training Accuracy', marker='o')
plt.plot(val_accuracies, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
torch.save(model.state_dict(), 'best_GoogLeNet_model_1.pth')

In [ ]:
from sklearn.metrics import accuracy_score
from torch.utils.data import Subset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import time

# Parameters
fractions = [0.2, 0.4,0.6, 0.8, 1.0]
num_epochs = 5
batch_size = 32
results = []

# Fix random seed
random.seed(42)
torch.manual_seed(42)


for frac in fractions:
    print(f"\nTraining on {int(frac * 100)}% of the dataset...")

    train_indices = list(range(len(augmented_train_data)))
    random.shuffle(train_indices)
    subset_len = int(frac * len(train_indices))
    subset_indices = train_indices[:subset_len]
    train_subset = Subset(augmented_train_data, subset_indices)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

    # Model
    model = models.googlenet(weights=GoogLeNet_Weights.DEFAULT)
    model.fc = nn.Sequential(nn.Dropout(p=0.5),nn.Linear(model.fc.in_features, num_classes))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    # Training
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Testing
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    results.append(acc)
    print(f"Test Accuracy with {int(frac * 100)}% data: {acc:.4f}")


plt.figure(figsize=(8, 5))
plt.plot([int(f*100) for f in fractions], results, marker='o')
plt.xlabel("Training Dataset Size (%)")
plt.ylabel("Test Accuracy")
plt.title("GoogLeNet - Test Accuracy vs Dataset Size")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
dataset_sizes = [20, 40, 60, 80, 100]
test_accuracies = [0.9929, 0.9982, 0.9965, 0.9982, 0.9911]



plt.figure(figsize=(8, 5))
plt.plot(dataset_sizes, test_accuracies, marker='o', linestyle='-', color='royalblue')

plt.xlabel("Training Dataset Size (%)")
plt.ylabel("Test Accuracy")
plt.title("Test Accuracy vs Dataset Size GoogLeNet)")
plt.ylim(0.9, 1)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=test_loader.dataset.dataset.classes
)
disp.plot(xticks_rotation=45)
plt.title("Confusion Matrix - Test Set")
plt.show()


## Tune 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import GoogLeNet_Weights
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import time

model = models.googlenet(weights=GoogLeNet_Weights.DEFAULT)
model.fc = nn.Sequential(nn.Dropout(p=0.5),nn.Linear(model.fc.in_features, num_classes))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.001)

train_accuracies = []
val_accuracies = []

start_time = time.time()
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_preds = []
    actual_labels = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

train_acc_google = np.array(train_accuracies)
val_acc_google = np.array(val_accuracies)

print(f"\nFinal Results:")
print(f"Average Training Accuracy over: {train_acc_google.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_google.mean():.4f}")

# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_accuracies, label='Training Accuracy', marker='o')
plt.plot(val_accuracies, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Tune 3

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import GoogLeNet_Weights
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import time

model = models.googlenet(weights=GoogLeNet_Weights.DEFAULT)
model.fc = nn.Sequential(nn.Dropout(p=0.5),nn.Linear(model.fc.in_features, num_classes))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0007, weight_decay = 0.00001)

train_accuracies = []
val_accuracies = []

start_time = time.time()
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_preds = []
    actual_labels = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(outputs.argmax(1).cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(actual_labels, train_preds)
    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.extend(outputs.argmax(1).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    avg_val_loss = val_loss / len(val_loader)

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} "
          f" accuracy: {train_acc:.4f} - loss: {avg_train_loss:.4f} "
          f"- val_accuracy: {val_acc:.4f} - val_loss: {avg_val_loss:.4f}")

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

train_acc_google = np.array(train_accuracies)
val_acc_google = np.array(val_accuracies)

print(f"\nFinal Results:")
print(f"Average Training Accuracy over: {train_acc_google.mean():.4f}")
print(f"Average Validation Accuracy over: {val_acc_google.mean():.4f}")

# Testing phase
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'Testing Accuracy: {accuracy:.4f}')
print(f'Testing Precision: {precision:.4f}')
print(f'Testing Recall: {recall:.4f}')
print(f'Testing F1 Score: {f1:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(10, 6))
plt.plot(train_accuracies, label='Training Accuracy', marker='o')
plt.plot(val_accuracies, label='Validation Accuracy', marker='s')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy over Epochs')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


#**Comparison**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


model_names = ['VGG16', 'ResNet18', 'DenseNet121', 'GoogLeNet']
val_accuracies = [0.9565, 0.9813, 0.9689, 0.973]
training_accuracy = [0.9426, 0.9888, 0.981, 0.9859]

x = np.arange(len(model_names))
width = 0.35

plt.figure(figsize=(10, 6))
bars1 = plt.bar(x - width/2, val_accuracies, width, label='Validation Accuracy')
bars2 = plt.bar(x + width/2, training_accuracy, width, label='Training Accuracy')




plt.xticks(x, model_names)
plt.ylim(0, 1.1)
plt.ylabel("Accuracy")
plt.title("Validation vs Training Accuracy for Best Models")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
#vgg16 graph

import matplotlib.pyplot as plt
import numpy as np


model_names = ['1st Combination', '2nd combination', '3rd combination']
val_accuracies = [0.9565, 0.7706, 0.9685]
training_accuracy = [0.9426, 0.7618, 0.9124]

x = np.arange(len(model_names))
width = 0.35

plt.figure(figsize=(10, 6))
bars1 = plt.bar(x - width/2, val_accuracies, width, label='Validation Accuracy')
bars2 = plt.bar(x + width/2, training_accuracy, width, label='Train Accuracy')



plt.xticks(x, model_names)
plt.ylim(0, 1.1)
plt.ylabel("Accuracy")
plt.title("Validation vs Training Accuracy for VGG16")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
#vgg16 graph

import matplotlib.pyplot as plt
import numpy as np


model_names = ['1st Combination', '2nd combination', '3rd combination']
val_accuracies = [0.9689, 0.9536, 0.9393]
training_accuracy = [0.981, 0.9731, 0.9789]

x = np.arange(len(model_names))
width = 0.35

plt.figure(figsize=(10, 6))
bars1 = plt.bar(x - width/2, val_accuracies, width, label='Validation Accuracy')
bars2 = plt.bar(x + width/2, training_accuracy, width, label='Train Accuracy')



plt.xticks(x, model_names)
plt.ylim(0, 1.1)
plt.ylabel("Accuracy")
plt.title("Validation vs Training Accuracy for DenseNet")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
#ResNet graph

import matplotlib.pyplot as plt
import numpy as np


model_names = ['1st Combination', '2nd combination', '3rd combination']
val_accuracies = [0.9813, 0.925, 0.946]
training_accuracy = [0.9888, 0.9831, 0.9817]

x = np.arange(len(model_names))
width = 0.35

plt.figure(figsize=(10, 6))
bars1 = plt.bar(x - width/2, val_accuracies, width, label='Validation Accuracy')
bars2 = plt.bar(x + width/2, training_accuracy, width, label='Train Accuracy')



plt.xticks(x, model_names)
plt.ylim(0, 1.1)
plt.ylabel("Accuracy")
plt.title("Validation vs Training Accuracy for ResNet")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
#GoogLeNet graph

import matplotlib.pyplot as plt
import numpy as np


model_names = ['1st Combination', '2nd combination', '3rd combination']
val_accuracies = [0.973, 0.9291, 0.9661]
training_accuracy = [0.9859, 0.9785, 0.9862]

x = np.arange(len(model_names))
width = 0.35

plt.figure(figsize=(10, 6))
bars1 = plt.bar(x - width/2, val_accuracies, width, label='Validation Accuracy')
bars2 = plt.bar(x + width/2, training_accuracy, width, label='Train Accuracy')



plt.xticks(x, model_names)
plt.ylim(0, 1.1)
plt.ylabel("Accuracy")
plt.title("Validation vs Training Accuracy for GoogLeNet")
plt.legend()
plt.tight_layout()
plt.show()


# Interface For Best Model

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image

In [ ]:
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torchvision.models import GoogLeNet_Weights
from PIL import Image

# --- CONFIGURATION ---
MODEL_PATH = "best_GoogLeNet_model_1.pth"
NUM_CLASSES = 4
CLASS_NAMES = ['Cloudy', 'Desert', 'Green Area', 'Water']

# --- MODEL WRAPPER ---
class GoogLeNetClassifier:
    def __init__(self, model_path, num_classes):
        self.model = models.googlenet(weights=GoogLeNet_Weights.DEFAULT)
        self.model.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(self.model.fc.in_features, num_classes)
        )
        self.model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
        self.model.eval()

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def predict(self, image):
        img_tensor = self.transform(image).unsqueeze(0)  # Shape: (1, 3, 224, 224)
        with torch.no_grad():
            output = self.model(img_tensor)
            probs = F.softmax(output, dim=1)
            class_idx = torch.argmax(probs, dim=1).item()
            confidence = probs[0][class_idx].item()
            return f"Predicted class: {CLASS_NAMES[class_idx]} (Confidence: {confidence:.2f})"

# --- INIT ---
classifier = GoogLeNetClassifier(MODEL_PATH, NUM_CLASSES)

# --- GRADIO UI ---
interface = gr.Interface(
    fn=classifier.predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="GoogLeNet Satellite Image Classifier",
    description="Upload a satellite image to classify it into one of four categories: Cloudy, Desert, Green Area, or Water."
)

interface.launch()
